In [ ]:
import pyvista as pv
import SimpleITK as sitk
import vtk
import numpy as np
from pathlib import Path
from meshpy.tet import MeshInfo, build
from scipy.ndimage import label, map_coordinates
import math
import os
import meshlib.mrmeshpy as mr
import meshlib.mrmeshnumpy as mrn

## Functions

In [ ]:
def numpy_to_vtk_coords(coords):
    """
    Convert an iterable of numeric coordinates to a vtk.vtkDoubleArray.

    Parameters
    ----------
    coords : iterable
        Sequence of numeric values (e.g., 1D list or array) to store in the
        vtkDoubleArray. Each entry is converted to float and stored in order.

    Returns
    -------
    vtk.vtkDoubleArray
        vtk array containing the provided values as doubles.
    """
    vtkarr = vtk.vtkDoubleArray()
    vtkarr.SetNumberOfValues(len(coords))  # allocate storage
    for i, v in enumerate(coords):
        vtkarr.SetValue(i, float(v))  # vtk expects native Python floats
    return vtkarr

def remove_small_islands(binary_matrix, area_threshold):
    """
    Remove small connected components from a binary mask in-place.

    Parameters
    ----------
    binary_matrix : numpy.ndarray
        3D boolean or integer array where connected components >0 are objects.
    area_threshold : int
        Components with a voxel count smaller than this threshold will be set to 0.

    Returns
    -------
    numpy.ndarray
        The modified binary_matrix with small components removed.
    """
    labeled_array, num_features = label(binary_matrix)  # connected-component labeling
    for i in range(1, num_features + 1):
        component = (labeled_array == i)
        if component.sum() < area_threshold:
            binary_matrix[component] = 0  # drop small islands
    return binary_matrix

def mask_region_of_interest(mask, ranges):
    """
    Return a mask where only the specified region (by slices) is preserved.

    Parameters
    ----------
    mask : numpy.ndarray
        Input 3D mask array.
    ranges : sequence of 6 ints
        Six values used as slice indices: [a0,a1,b0,b1,c0,c1]. These are applied
        as mask[a0:a1, b0:b1, c0:c1] (the same ordering as used in the code).

    Returns
    -------
    numpy.ndarray
        New mask with all voxels outside the specified region set to 0.
    """
    zero_mask = np.zeros(np.shape(mask))
    # ranges indices are applied in the same ordering used elsewhere in this notebook
    zero_mask[ranges[0]:ranges[1], ranges[2]:ranges[3], ranges[4]:ranges[5]] = \
        mask[ranges[0]:ranges[1], ranges[2]:ranges[3], ranges[4]:ranges[5]]
    return zero_mask

def create_blocks(mask, thickness, axis):
    """
    Add block regions (value 2) at the two opposing faces of the object along
    the given axis and return the combined mask.

    Parameters
    ----------
    mask : numpy.ndarray
        Input 3D mask array (assumed binary where object voxels == 1).
    thickness : int
        Thickness in voxels for the added block regions at the object ends.
    axis : str
        Axis along which to create blocks. Currently only 'X' is implemented.

    Returns
    -------
    numpy.ndarray
        Mask with added blocks (value 2) combined with the original mask.
    """
    if (axis == "X"):
        # find object extents along X and compute Y,Z extents near the faces
        xmin = np.min(np.where(mask == 1)[0])
        xmax = np.max(np.where(mask == 1)[0]) + 1
        ymin = np.min([np.min(np.where(mask[xmin:xmin+thickness, :, :] == 1)[1]),
                       np.min(np.where(mask[xmax-thickness:xmax, :, :] == 1)[1])])
        ymax = np.max([np.max(np.where(mask[xmin:xmin+thickness, :, :] == 1)[1]),
                       np.max(np.where(mask[xmax-thickness:xmax, :, :] == 1)[1])]) + 1
        zmin = np.min([np.min(np.where(mask[xmin:xmin+thickness, :, :] == 1)[2]),
                       np.min(np.where(mask[xmax-thickness:xmax, :, :] == 1)[2])])
        zmax = np.max([np.max(np.where(mask[xmin:xmin+thickness, :, :] == 1)[2]),
                       np.max(np.where(mask[xmax-thickness:xmax, :, :] == 1)[2])]) + 1
        blocks = np.zeros(np.shape(mask))
        blocks[xmin:xmin+thickness, ymin:ymax, zmin:zmax] = 2  # add block at lower X face
        blocks[xmax-thickness:xmax, ymin:ymax, zmin:zmax] = 2  # add block at upper X face
        final_mask = blocks + mask  # preserve original mask values (1) and add blocks (2)
        return final_mask

def _skew(v):
    """
    Return the 3x3 skew-symmetric matrix for vector v used in Rodrigues' formula.

    Parameters
    ----------
    v : array-like, shape (3,)
        Input 3D vector.

    Returns
    -------
    numpy.ndarray
        3x3 skew-symmetric matrix.
    """
    # used to compute cross-product matrix K such that K @ x = v x x
    return np.array([[0.0, -v[2], v[1]],
                     [v[2],  0.0, -v[0]],
                     [-v[1], v[0],  0.0]], dtype=float)

def align_by_centroids_roi(image, iso_value, roi=[0,0,0,0,0,0], fraction=0.1, axis="Z", debug=True):
    """
    Align a SimpleITK volume so the line connecting top and bottom centroids
    (computed inside the ROI at the given threshold) is aligned to a target axis.

    Parameters
    ----------
    image : SimpleITK.Image
        Input volume.
    iso_value : numeric
        Intensity threshold used to compute the binary mask for centroid estimation.
    roi : list of 6 ints
        Slice indices used to extract the region where centroids are computed.
        Ordered as [x0,x1,y0,y1,z0,z1] when calling this function.
    fraction : float
        Fraction of slices from each end of the ROI used to compute top/bottom centroids.
    axis : {'X','Y','Z'}
        Target world axis to align the centroid vector to.
    debug : bool
        If True, print diagnostic information and the rotation matrix.

    Returns
    -------
    tuple
        (out_arr, out) where out_arr is the numpy array (z,y,x) of the rotated image
        and out is the rotated SimpleITK.Image object (resampled to fit).
    """
    arr = sitk.GetArrayFromImage(image)  # convert ITK image to numpy (z,y,x)

    print(roi)  # show ROI used for debugging

    x0, x1, y0, y1, z0, z1 = roi
    if np.prod(roi) != 0:
        roi_arr = arr[z0:z1, y0:y1, x0:x1]  # note ordering: z,y,x slices
    else:
        roi_arr = arr
    mask = roi_arr > iso_value  # binary mask inside ROI

    nz, ny, nx = mask.shape
    if nz <= 0 or ny <= 0 or nx <= 0:
        raise ValueError("ROI is empty or invalid")

    n_slices = max(1, int(nz * fraction))  # number of slices from each end to use

    bottom_mask = mask[:n_slices]
    if bottom_mask.sum() == 0:
        raise ValueError("No voxels in the bottom ROI slices. Decrease fraction or change ROI.")
    bottom_coords = np.argwhere(bottom_mask)
    bottom_centroid_roi = bottom_coords.mean(axis=0)  # centroid in ROI-local z,y,x coords

    top_mask = mask[-n_slices:]
    if top_mask.sum() == 0:
        raise ValueError("No voxels in the top ROI slices. Decrease fraction or change ROI.")
    top_coords = np.argwhere(top_mask)
    top_coords[:, 0] += nz - n_slices  # shift top coords to ROI local indexing
    top_centroid_roi = top_coords.mean(axis=0)

    # convert ROI-local centroids to full-image z,y,x indices
    bottom_idx_zyx = bottom_centroid_roi + np.array([z0, y0, x0])
    top_idx_zyx    = top_centroid_roi    + np.array([z0, y0, x0])

    # get image geometry
    spacing = np.array(image.GetSpacing(), dtype=float)
    origin  = np.array(image.GetOrigin(), dtype=float)
    D = np.array(image.GetDirection(), dtype=float).reshape(3,3)  # direction cosines
    M = D @ np.diag(spacing)  # index -> physical: p = origin + M @ index_xyz

    # convert z,y,x -> x,y,z index convention for matrix multiplication
    bottom_idx_xyz = bottom_idx_zyx[::-1]
    top_idx_xyz    = top_idx_zyx[::-1]

    bottom_phys = origin + M @ bottom_idx_xyz  # physical coordinates in mm
    top_phys    = origin + M @ top_idx_xyz

    v = top_phys - bottom_phys  # vector from bottom to top in physical space
    norm_v = np.linalg.norm(v)
    if norm_v == 0:
        raise ValueError("Top and bottom centroids coincide (zero-length). Choose different ROI/fraction.")
    v_unit = v / norm_v

    axis = axis.upper()
    if axis == "X":
        t = np.array([1.0, 0.0, 0.0], dtype=float)
    elif axis == "Y":
        t = np.array([0.0, 1.0, 0.0], dtype=float)
    else:
        t = np.array([0.0, 0.0, 1.0], dtype=float)

    # Rodrigues rotation: compute axis (k), sine (s) and cosine (c)
    k = np.cross(v_unit, t)
    s = np.linalg.norm(k)
    c = np.dot(v_unit, t)

    if s < 1e-12 and c > 0.999999:
        R = np.eye(3)  # already aligned
    elif s < 1e-12 and c < -0.999999:
        # 180-degree rotation: pick an orthogonal axis
        arbitrary = np.array([1.0, 0.0, 0.0], dtype=float)
        if abs(np.dot(arbitrary, v_unit)) > 0.9:
            arbitrary = np.array([0.0, 1.0, 0.0], dtype=float)
        axis_orth = np.cross(v_unit, arbitrary)
        axis_orth /= np.linalg.norm(axis_orth)
        K = _skew(axis_orth)
        R = np.eye(3) + 2.0 * (K @ K)  # rotation by pi
    else:
        K = _skew(k)
        # R = I + K + K^2 * ((1 - c) / (s^2))  # Rodrigues' formula
        R = np.eye(3) + K + (K @ K) * ((1.0 - c) / (s * s))

    if debug:
        ang = math.degrees(math.acos(np.clip(c, -1.0, 1.0)))
        print("bottom_idx_zyx (full image):", bottom_idx_zyx)
        print("top_idx_zyx    (full image):", top_idx_zyx)
        print("bottom_phys (mm):", bottom_phys)
        print("top_phys    (mm):", top_phys)
        print(f"Angle between centroid vector and target axis: {ang:.6f} deg")
        print("Rotation matrix R:\n", R)

    # Build output reference grid so rotated volume fits entirely
    size = np.array(image.GetSize(), dtype=float)
    corners_idx_xyz = np.array([[0,0,0],
                                [size[0]-1,0,0],
                                [0,size[1]-1,0],
                                [0,0,size[2]-1],
                                [size[0]-1,size[1]-1,0],
                                [size[0]-1,0,size[2]-1],
                                [0,size[1]-1,size[2]-1],
                                [size[0]-1,size[1]-1,size[2]-1]], dtype=float)
    corners_phys = (corners_idx_xyz @ M.T) + origin  # physical coords of corners

    center_idx_xyz = (size - 1.0) / 2.0
    center_phys = origin + M @ center_idx_xyz  # geometric center in physical space

    rotated_corners = ((corners_phys - center_phys) @ R.T) + center_phys

    out_spacing = spacing.copy()
    mins = rotated_corners.min(axis=0)
    maxs = rotated_corners.max(axis=0)
    extent = maxs - mins
    out_size = np.ceil(extent / out_spacing).astype(int)
    out_size = np.maximum(out_size, 1)
    out_origin = mins

    ref = sitk.Image(int(out_size[0]), int(out_size[1]), int(out_size[2]), image.GetPixelID())
    ref.SetSpacing(tuple(float(x) for x in out_spacing))
    ref.SetOrigin(tuple(float(x) for x in out_origin))
    ref.SetDirection(tuple(np.eye(3).ravel().tolist()))  # keep identity direction for output

    R_inv = R.T
    T = sitk.AffineTransform(3)
    T.SetMatrix(list(R_inv.ravel()))  # transform maps output->input indices
    T.SetCenter(tuple(center_phys.tolist()))

    out = sitk.Resample(
        image,
        ref,
        T,
        sitk.sitkNearestNeighbor,  # nearest to preserve discrete densities
        0.0,
        image.GetPixelID()
    )

    out_arr = sitk.GetArrayFromImage(out)
    if debug:
        print("Output image shape (z,y,x):", out_arr.shape,
              " intensity range:", out_arr.min(), out_arr.max())
    return out_arr, out

## Import the .nddr CT file

In [ ]:
input_nrrd = "microCT_volume_preview045423.nrrd"  # input CT file
iso_value = 8000  # intensity threshold for segmentation
downsample_factor = 4  # integer factor to reduce voxel counts for speed
roi = [0,70,0,70,0,140]  # ROI slices (x0,x1,y0,y1,z0,z1)

if not os.path.exists(input_nrrd):
    raise FileNotFoundError(f"File not found: {input_nrrd}")

image = sitk.ReadImage(input_nrrd)  # SimpleITK image object

# align volume using centroids computed inside the ROI (returns numpy array and rotated SITK image)
array, rot_im = align_by_centroids_roi(image, iso_value, roi, fraction=0.1, axis="Z", debug=True)
image = rot_im

orig_size = np.array(image.GetSize())  # size in ITK order (sx,sy,sz)
orig_spacing = np.array(image.GetSpacing())  # spacing in mm
origin = np.array(image.GetOrigin())

# compute new size/spacing for downsampling (keep physical extent)
new_size = (orig_size / downsample_factor).astype(int)
new_spacing = orig_spacing * (orig_size / new_size)

resampler = sitk.ResampleImageFilter()
resampler.SetSize([int(s) for s in new_size])
resampler.SetOutputSpacing([float(s) for s in new_spacing])
resampler.SetOutputOrigin(origin)
resampler.SetOutputDirection(image.GetDirection())
resampler.SetInterpolator(sitk.sitkLinear)
image = resampler.Execute(image)  # resampled SITK image

# convert to numpy array: SimpleITK uses (z,y,x) ordering
array = sitk.GetArrayFromImage(image)
spacing = np.array(image.GetSpacing())
origin = np.array(image.GetOrigin())
print(f"Loaded image: shape={array.shape}, intensity range=({array.min()}, {array.max()})")

# apply threshold to produce a binary mask
mask = array >= iso_value
n_voxels = np.sum(mask)
print(f"Threshold applied: {n_voxels} voxels above threshold")
if n_voxels == 0:
    raise ValueError("No voxels above threshold. Reduce iso_value.")

# clean small isolated components and restrict to ROI and add support blocks
mask = remove_small_islands(mask, 30)
mask = mask_region_of_interest(mask, roi)
mask = create_blocks(mask, 5, "X")

# convert boolean mask to mrmesh simpleVolume and create a surface mesh (STL)
simpleVolume = mrn.simpleVolumeFrom3Darray(np.float32(mask > 0))
floatGrid = mr.simpleVolumeToDenseGrid(simpleVolume)
mesh_stl = mr.gridToMesh(floatGrid, mr.Vector3f(1.0, 1.0, 1.0), 0.5)  # grid->mesh using isovalue 0.5
stl_path = Path(input_nrrd).stem + "_TETmesh.stl"
mr.saveMesh(mesh_stl, stl_path)

# read mesh with pyvista and optionally decimate to reduce triangle count
mesh_nuclei = pv.read(stl_path)
if mesh_nuclei.volume > 0.0:
    mesh_nuclei.decimate(target_reduction=0.8, inplace=True)

# prepare surface for tetrahedralization
surface_mesh = pv.read(stl_path)
points = np.array(surface_mesh.points)
faces = surface_mesh.faces.reshape(-1, 4)[:, 1:]  # convert pyvista faces to Nx3 array

mesh_info = MeshInfo()
mesh_info.set_points(points)
mesh_info.set_facets(faces.tolist())

# build tetrahedral mesh with meshpy (max_volume controls element size)
tet_mesh = build(mesh_info, max_volume=1.0)

tet_points = np.array(tet_mesh.points)
# meshpy returns elements as tuples of zero-based node indices; prep for pyvista UnstructuredGrid
tet_elements = []
for tet in tet_mesh.elements:
    tet_elements.extend([4, *tet])  # prepend cell node count (4) for VTK format
tet_elements = np.array(tet_elements)
celltypes = np.full(len(tet_mesh.elements), pv.CellType.TETRA, dtype=np.uint8)

grid = pv.UnstructuredGrid(tet_elements, celltypes, tet_points)

print("\n--- Coordinate Sanity Check ---")
print("Voxel index limits (z,y,x):", array.shape)
print("Centroid range:", tet_points.min(axis=0), tet_points.max(axis=0))

# sample density and compute Young's modulus per tetra element
cell_densities = []
cell_ymodulus = []
for tet in tet_mesh.elements:
    centroid = tet_points[tet].mean(axis=0)
    v_coord = centroid[[0, 1, 2]]  # centroid in x,y,z physical or index space depending on mesh generation
    # round coordinates to nearest voxel index for mask lookup
    if mask[round(v_coord[0]), round(v_coord[1]), round(v_coord[2])] < 2:
        density_val = map_coordinates(array, v_coord.reshape(3, 1), order=1, mode='nearest')
        # convert sampled intensity to density using an empirical formula
        density_val = (density_val[0] * 3.85) / 6.6e4
        ymodulus_val = (density_val ** 2.0) * 1.4e4
        cell_densities.append(density_val)
        cell_ymodulus.append(ymodulus_val)
    else:
        # fallback values for block/support regions
        cell_densities.append(5.0)
        cell_ymodulus.append(100000)

# attach cell data for visualization / export
grid.cell_data["Density (g/cm3)"] = np.array(cell_densities)
grid.cell_data["YM (MPa)"] = np.array(cell_ymodulus)

# save unstructured grid to VTK
vtk_path = Path(input_nrrd).stem + "_TETmesh_rot.vtk"
grid.save(vtk_path)
print(f"Tetrahedral mesh with densities saved to {vtk_path}")

In [ ]:
from collections import defaultdict  # for potential grouping helpers

tolerance = 0.01  # relative tolerance for grouping E values (1%)
# sort elements by E value (element id, E)
sorted_elements = sorted(
    [(i + 1, E) for i, E in enumerate(cell_ymodulus)],
    key=lambda x: x[1]
)

E_groups = []  # list of lists of element ids per group
group_values = []  # representative E value per group (updated as average)

# simple greedy clustering: iterate sorted elements and assign to first compatible group
for elem_id, E in sorted_elements:
    placed = False
    for idx, g_val in enumerate(group_values):
        # relative difference to group representative
        if abs(E - g_val) / g_val <= tolerance:
            E_groups[idx].append(elem_id)
            # update group representative as running average to keep cluster centered
            group_values[idx] = (group_values[idx] * (len(E_groups[idx]) - 1) + E) / len(E_groups[idx])
            placed = True
            break
    if not placed:
        # create new group when no existing group within tolerance
        group_values.append(E)
        E_groups.append([elem_id])

In [ ]:
inp_path = Path(input_nrrd).stem + "_TETmesh_binned_E.inp"
with open(inp_path, "w") as f:
    f.write("*Heading\n")
    f.write("** Generated by Python script\n")
    
    # write node coordinates (Abaqus expects node index, X, Y, Z)
    f.write("*Node\n")
    for i, p in enumerate(tet_points, start=1):
        f.write(f"{i}, {p[0]:.6f}, {p[1]:.6f}, {p[2]:.6f}\n")
    
    # write tetrahedral elements (C3D4 element type)
    f.write("*Element, type=C3D4\n")
    for i, e in enumerate(tet_mesh.elements, start=1):
        # elements from meshpy are zero-based indices; Abaqus expects 1-based node ids
        f.write(f"{i}, {e[0]+1}, {e[1]+1}, {e[2]+1}, {e[3]+1}\n")

    # write material/elset sections for each E-group
    for mat_idx, (E_val, elems) in enumerate(zip(group_values, E_groups), start=1):
        f.write(f"*Elset, elset=ESET{mat_idx}\n")
        for e_id in elems:
            f.write(f"{e_id},\n")
        
        f.write(f"*Material, name=MAT{mat_idx}\n")
        f.write("*Elastic\n")
        # write Young's modulus and Poisson's ratio
        f.write(f"{E_val:.6f}, 0.3\n")
        
        # assign section linking element set to material
        f.write(f"*Solid Section, elset=ESET{mat_idx}, material=MAT{mat_idx}\n")